<a href="https://colab.research.google.com/github/emilycalvert/scripts_cite/blob/main/verficar_tallando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alistar entorno.

In [12]:
!pip install numpy
!pip install geopy
!pip install pandas
!pip install pyproj
!pip install timedelta

In [13]:
import ee
import time
import folium
import datetime
import numpy as np
import pandas as pd
from folium import plugins
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
from pyproj import Transformer, Proj, transform

In [14]:
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=TNyoqjq9_l41
# Add custom basemaps to folium
basemaps = {
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    )
}

In [15]:
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=TNyoqjq9_l41

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# **1. Autorizar e inicializar Earth Engine**
#### _Autenticar e inicializar la API de Google Earth Engine (GEE)._




- Conecte e inicialice con tu cuenta de earth engine y proyecto.

In [ ]:
ee.Authenticate(auth_mode="notebook")
ee.Initialize()

# **2. Cargar y analizar CSV**

#### _Cargue el archivo CSV que contiene las coordenadas y analícelo para crear puntos._

- Exporta tu hoja de cálculo con los vértices como csv. La integridad y uniformidad del documento original es importante. Si tiene dificultades en este paso, consulte el curso [aquí.](https://www.youtube.com/watch?v=uGx0PHD6o9M)

- Al usar este cuaderno en colab. Habrá un panel a la izquierda. Si no estás en los archivos. Haga clic en el icono de la carpeta donde los botones están al ras de la pantalla. Ahora donde dice "Archivos", haz clic en el ícono de archivo con una flecha en el medio y sube tus vértices. Alternativamente, puede arrastrar y soltar en el panel de archivos.

# WHEN THE SCRIPT IS DONE DELETE THE COMMENTED OUT CELLS


In [12]:
csv_viaje = 'imazacoord.csv'  # Una vez que haya cargado su archivo, en esta línea cambie el nombre por el nombre del archivo que cargó con la extensión.
df = pd.read_csv(csv_viaje)

In [13]:
# Aquí necesitamos proyectar desde UTM 18S a WGS84 (el CRS utilizado por Earth Engine).
transformer = Transformer.from_crs("epsg:32718", "epsg:4326", always_xy=True)
'''
Tenga en cuenta que si sus coordenadas no están en UTM 18S,
deberá consultar la documentación de pyproj y ajustar "epsg:32718"
al epsg que utilizan sus coordenadas. Aquí está el sitio donde
puedes encontrar esa información:
https://pyproj4.github.io/pyproj/stable/api/crs/crs.html
https://epsg.io/
'''
def utm_to_latlon(easting, northing):
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

df['latitud'], df['longitud'] = zip(*df.apply(lambda row: utm_to_latlon(row['ESTE'], row['NORTE']), axis=1)) # Donde dice 'ESTE' y 'NORTE', esto coincide de manera muy idéntica con los nombres de las columnas de latitud y longitud en sus datos.

In [ ]:
# df.head() # Ahora puedes verificar tus operaciones.

In [ ]:
coordenadas = df[['latitud', 'longitud']].values.tolist()

In [ ]:
# coordenadas[0]

In [ ]:
puntos = [ee.Feature(ee.Geometry.Point(lon, lat)) for lat, lon in coordenadas]
puntos_fc = ee.FeatureCollection(puntos)

In [ ]:
# puntos_fc

In [ ]:
polygon = ee.Geometry.Polygon(coordenadas)

In [ ]:
polygon_geojson = ee.FeatureCollection([ee.Feature(polygon)]).getInfo()

In [ ]:
# display(polygon)

In [ ]:
centroid = polygon.centroid(maxError=1)

In [ ]:
centroid.getInfo()

In [ ]:
centro= (-4.349571452252664, -73.72755930052607) # Aquí pegue las coordenadas impresas en la celda de arriba para definir el centro de su parcela.

In [ ]:
# Fetch the most recent Landsat image
'''
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
          .filterDate('2013-01-01', '2024-02-10') \
          .filterBounds(puntos_fc) \
          .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
          .sort('system:time_start', False).first()  # False to sort in descending order
'''

In [ ]:
# print(landsat.getInfo())

In [ ]:
# landsat.getMapId()

In [ ]:
'''
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=TNyoqjq9_l41

# Set visualization parameters.
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}


# Create a folium map object., '%Y-%m-%d')
my_map = folium.Map(location=[centro[0], centro[1]], zoom_start=15)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(landsat, vis_params, 'Landsat 8 Image')

# Add the polygon to the map using Folium's GeoJson method for custom styling.
folium.GeoJson(polygon_geojson).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)
'''

# **3. Obtener imágenes de Landsat:**

#### _Utilice la API de Earth Engine para consultar imágenes de Landsat 8 para las fechas y la cobertura de nubes especificadas._

In [ ]:
antes = '2023-01-06'
'''
Establezca esto en una fecha anterior a la emisión del permiso.
Encontrará la primera imagen de satélite en esa fecha o antes que
cumpla con sus requisitos de cobertura de nubes.
'''
despues = '2024-01-06'
'''
Esta fecha es posterior a la cosecha y encontrará
una imagen para la fecha más cercana pero no anterior
a la cosecha que cumpla con el umbral de nubosidad.
'''
limite_coberatura_nubes = 15  # Establezca esto en su umbral máximo de cobertura de nubes.

In [ ]:
# Imagen antes de la emisión del permiso.
# Un día, en un futuro lejano, Landsat 8 podría retirarse por 9. En este caso, será necesario corregir el script.
landsat1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
           .filterDate(antes) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUD_COVER', limite_coberatura_nubes)) \
           .sort(antes, ascending= False).first()

ndvi1 = landsat1.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

In [ ]:
# Aquí está la foto después de la cosecha.
landsat2 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
           .filterDate(despues) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUD_COVER', limite_coberatura_nubes)) \
           .sort('system:time_start', True).first()

ndvi2 = landsat2.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

In [ ]:
# Aquí vemos la trama previa a la emisión del permiso.
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=TNyoqjq9_l41

vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

my_map = folium.Map(location=[centro[0], centro[1]], zoom_start=15)

basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

my_map.add_ee_layer(landsat1, vis_params, 'Landsat 8 Image')

folium.GeoJson(polygon_geojson).add_to(my_map)

my_map.add_child(folium.LayerControl())

plugins.Fullscreen().add_to(my_map)

display(my_map)

# **4. Calcular NDVI:**
#### _Calcular y enmascarar áreas donde se excede el umbral._

In [ ]:
ndvi_diff = ndvi2.subtract(ndvi1).rename('Change')
threshold = 0.1
'''
Aquí es posible que deba ajustar el umbral.
Es importante monitorear para asegurarse de que el umbral dado
refleje con precisión el área cosechada.
'''
diff_mask = ndvi_diff.gt(threshold)

In [ ]:
diff_mask

In [ ]:
vectors = diff_mask.reduceToVectors({
    'geometryType': 'polygon',
    'reducer': ee.Reducer.countEvery(),
    'scale': 30, # Esto debe ajustarse según el tamaño de su paquete y la resolución de las imágenes.
    'maxPixels': 1e8
})

In [ ]:
geojson_output = vectors.getInfo()

# **5. Visualizar**
#### _Muestra los polígonos identificados sobre la imagen Landsat más reciente._

In [ ]:
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

my_map = folium.Map(location=[centro[0], centro[1]], zoom_start=15)

basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

my_map.add_ee_layer(landsat2, vis_params, 'Landsat 8 Image')

folium.GeoJson(geojson_output).add_to(my_map)

my_map.add_child(folium.LayerControl())

plugins.Fullscreen().add_to(my_map)

display(my_map)

In [ ]:
# Estos son los vértices de los polígonos donde hubo cosecha.
def print_polygon_vertices(geojson):
    for i, feature in enumerate(geojson['features']):
        if 'geometry' in feature and 'coordinates' in feature['geometry']:
            polygon_coordinates = feature['geometry']['coordinates'][0]

            print(f"Title Here: Polygon {i + 1}")

            print(f"{'Vertex':<10}{'Latitude':<20}{'Longitude':<20}")

            for j, coord in enumerate(polygon_coordinates):
                lat, lon = coord
                print(f"{j + 1:<10}{lat:<20}{lon:<20}")

            print("\n" + "-"*50 + "\n")

print_polygon_vertices(geojson_output)